In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import folium
import googlemaps

In [2]:
sum1=30

In [3]:

driver=webdriver.Chrome(service=Service("driver/chromedriver"))
driver.get("https://www.tripadvisor.co.kr/Search?q=%EC%84%9C%EC%9A%B8%EC%8B%9C&searchSessionId=5E26E3232FEC6BC3D2EA51D1ABD7BBBD1648786205597ssid&searchNearby=false&geo=294197&sid=F2200CA8D89747D0BE0CF663271D5F631648786221283&blockRedirect=true&ssrc=e&rf=5")
time.sleep(3)
soup=BeautifulSoup(driver.page_source,"lxml")
title_list=soup.select("div.result-title>span")
title_list1=[]
for n in title_list:
    title_list1.append(n.text)
address_list=soup.select("div.address>div.address-text")
address_list1=[]
for n in address_list:
    address_list1.append(n.text)
# 뒤에 driver.get에 url 보면 (9,30),(10,60),(11,61)이렇게 증가함.
for i in range(9,43):
    driver.get("https://www.tripadvisor.co.kr/Search?q=%EC%84%9C%EC%9A%B8%EC%8B%9C&searchSessionId=5E26E3232FEC6BC3D2EA51D1ABD7BBBD1648786205597ssid&searchNearby=false&geo=294197&sid=F2200CA8D89747D0BE0CF663271D5F631648786221283&blockRedirect=true&ssrc=e&rf="+str(i)+"&o="+str(sum1))
#     url이 다 켜져야되기때문에 이렇게 3초 기간을 주었음.
    time.sleep(3)
    soup=BeautifulSoup(driver.page_source,"lxml")
    title_list=soup.select("div.result-title>span")
    for n in title_list:
        title_list1.append(n.text)
    address_list=soup.select("div.address>div.address-text")
    for n in address_list:
        address_list1.append(n.text)
    sum1+=30
driver.close()

In [4]:
data={"가게이름":title_list1,"주소":address_list1}
df=pd.DataFrame(data)

,가게이름,주소
0,아리아,"중구 소공로 106, 서울, 대한민국"
1,스시로로,"동작대로23길 29, 서울, 대한민국"
2,롯데리아 롯데마트서울역사점,"중구 청파로 426, 서울, 대한민국"
3,브루클린 더버거조인트,"서초구 동광로43길 51-13, 서울, 대한민국"
4,모모야마,"중구 을지로 30, 서울, 대한민국"
...,...,...
1045,부산돼지국밥,"남대문시장길 29, 서울, 대한민국"
1046,아끼야,"서초중앙로20길 14, 서울, 대한민국"
1047,둘세이수아베,"도산대로11길 32, 서울, 대한민국"
1048,장터빈대떡,"겸재로 212, 서울, 대한민국"


In [5]:
f = open("key/진호key.txt", 'r')
gmaps_key =f.readlines()
gmaps = googlemaps.Client(key=gmaps_key[0])

In [ ]:
lat=list()
lng=list()
for n in tqdm(df.index):
        target_name=df['주소'][n]+", " +"South Korea"
        gmaps_output=gmaps.geocode(target_name)
        time.sleep(0.5)
        location=gmaps_output[0].get("geometry")
        lat.append(location['location']['lat'])
        lng.append(location['location']['lng'])

  0%|          | 0/1050 [00:00<?, ?it/s]

In [8]:
df['lat']=lat
df['lng']=lng

In [9]:
map=folium.Map(location=[37.5502,126.982],
              zoom_start=8)  
# icon_image1 = 'image/joyfree_03.png'

for n in df.index:
    folium.Marker(
        location=[df['lat'][n],df['lng'][n]],
        tooltip=df['가게이름'][n],
        popup='<pre>' + df["주소"][n]+"</pre>"
    ).add_to(map)

map

In [20]:
df

,가게이름,주소,lat,lng
0,아리아,"중구 소공로 106, 서울, 대한민국",37.564406,126.979697
1,스시로로,"동작대로23길 29, 서울, 대한민국",37.484054,126.980094
2,롯데리아 롯데마트서울역사점,"중구 청파로 426, 서울, 대한민국",37.557407,126.969500
3,브루클린 더버거조인트,"서초구 동광로43길 51-13, 서울, 대한민국",37.497211,127.000680
4,모모야마,"중구 을지로 30, 서울, 대한민국",37.564990,126.981369
...,...,...,...,...
1015,부산돼지국밥,"남대문시장길 29, 서울, 대한민국",37.560419,126.977624
1016,아끼야,"서초중앙로20길 14, 서울, 대한민국",37.492138,127.014830
1017,둘세이수아베,"도산대로11길 32, 서울, 대한민국",37.519567,127.022557
1018,장터빈대떡,"겸재로 212, 서울, 대한민국",37.588972,127.091197
